In [ ]:
import numpy as np
import mne
from sklearn.preprocessing import MinMaxScaler
from tensorflow import keras
from keras.models import Model
from keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from keras.optimizers import RMSprop


def load_physionet_data(subject_ids, channels):
    raw_files = []
    for subject in subject_ids:
        raw = mne.io.read_raw_edf(f'files/S{subject:03d}/S{subject:03d}R01.edf', preload=True)
        raw.pick_channels(channels)
        raw_files.append(raw)
    return raw_files


def preprocess_data(raw, T=160, delta=4, Delta=8):

    data = raw.get_data().T 
    scaler = MinMaxScaler()
    normalized_data = scaler.fit_transform(data)
    

    n_samples = normalized_data.shape[0]
    windows = []
    for start in range(0, n_samples - T + 1, delta):
        window = normalized_data[start:start+T, :]
        windows.append(window)
    return np.array(windows)


def augment_data(windows, Gamma=236, Delta=8):
    augmented = []
    for i in range(0, len(windows) - Gamma + 1, Delta):
        augmented.append(windows[i:i+Gamma])
    return np.array(augmented)


channels = ['Oz..', 'T7..', 'Cz..']
alpha_subjects = list(range(1, 91))
raw_data = load_physionet_data(alpha_subjects, channels)
windows = [preprocess_data(raw) for raw in raw_data]
augmented = [augment_data(win) for win in windows]
X_train = np.concatenate(augmented, axis=0)
y_train = np.repeat(np.arange(len(alpha_subjects)), [len(a) for a in augmented])

Extracting EDF parameters from /Users/kris/Desktop/COGS189/COGS189_FinalProject/files/S001/S001R01.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 9759  =      0.000 ...    60.994 secs...
Extracting EDF parameters from /Users/kris/Desktop/COGS189/COGS189_FinalProject/files/S002/S002R01.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 9759  =      0.000 ...    60.994 secs...
Extracting EDF parameters from /Users/kris/Desktop/COGS189/COGS189_FinalProject/files/S003/S003R01.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 9759  =      0.000 ...    60.994 secs...
Extracting EDF parameters from /Users/kris/Desktop/COGS189/COGS189_FinalProject/files/S004/S004R01.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 9759  =      0.000 ...    60.994 secs...
Extracting EDF parameter

: 

In [ ]:
def build_classification_model(input_shape, n_classes):
    inputs = Input(shape=input_shape)
    x = Conv2D(128, (3,3), activation='relu')(inputs)
    x = MaxPooling2D((2,2))(x)
    x = Conv2D(256, (3,3), activation='relu')(x)
    x = MaxPooling2D((2,2))(x)
    x = Conv2D(512, (3,3), activation='relu')(x)
    x = Flatten()(x)
    x = Dense(1024, activation='relu')(x)
    x = Dropout(0.25)(x)
    outputs = Dense(n_classes, activation='softmax')(x)
    
    model = Model(inputs, outputs)
    model.compile(optimizer=RMSprop(0.0001), 
                  loss='sparse_categorical_crossentropy', 
                  metrics=['accuracy'])
    return model


model = build_classification_model((20, 160, 3), len(alpha_subjects))
model.fit(X_train, y_train, epochs=30, batch_size=64)


fingerprint_model = Model(inputs=model.input, outputs=model.layers[-4].output)

In [ ]:
from sklearn.metrics import pairwise_distances

def enroll_user(eeg_sample):
    fingerprint = fingerprint_model.predict(eeg_sample)
    return fingerprint.mean(axis=0) 

def authenticate(claimed_fingerprint, new_sample, threshold=0.275):
    new_fingerprint = fingerprint_model.predict(new_sample).mean(axis=0)
    distance = pairwise_distances([claimed_fingerprint], [new_fingerprint], metric='cosine')[0][0]
    return distance <= threshold


user_sample = X_train[0]
stored_fingerprint = enroll_user(user_sample)


new_sample = X_train[1]
is_authenticated = authenticate(stored_fingerprint, new_sample)
print("Authentication result:", is_authenticated)